https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


In [2]:
import pandas as pd
import numpy as np
import pickle
import re


import warnings
warnings.filterwarnings('ignore')

In [3]:
with open('./with_name/LDA_merged_full.pkl', 'rb') as f:
    corpus = pickle.load(f)

In [ ]:
# with open('./with_name/----.pkl', 'wb') as f:
#     pickle.dump(corpus, f)

In [4]:
corpus

,뉴스제목,주가변동,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,LG생활건강 숨° 로시크숨마 황제의 빛 에디션 선봬,1,0,1,0.9640,"[(0, 0.035994086), (1, 0.96400595)]"
1,IRSK이노베이션 내년 시설투자 올해보다 낮은 수준,1,1,0,0.9440,"[(0, 0.94404787), (1, 0.055952135)]"
2,수출입銀 CJ대한통운 해외채권 발행 공동 보증,0,2,0,0.6790,"[(0, 0.6790463), (1, 0.32095373)]"
3,브랜드 아파트 만원대에 마련할 마지막 기회 사천 KCC스위첸,1,3,0,0.9012,"[(0, 0.90122193), (1, 0.09877808)]"
4,LG전자 뉴욕 한복판에서 영화 상영한 사연,0,4,1,0.9518,"[(0, 0.048231106), (1, 0.9517689)]"
...,...,...,...,...,...,...
611468,한국항공우주 인니에 억원 규모 KT B 훈련기 공급 계약,1,611468,0,0.8536,"[(0, 0.8535616), (1, 0.14643842)]"
611469,하이트진로 세 박태영 사장 승진,1,611469,0,0.7068,"[(0, 0.7068181), (1, 0.2931819)]"
611470,특징주 형제의 난 재점화 가능성에롯데지주 강세,0,611470,1,0.8101,"[(0, 0.1899097), (1, 0.8100903)]"
611471,제일기획 비상교육과 청소년 대상 마음 교과서 개발,1,611471,0,0.7944,"[(0, 0.79441136), (1, 0.2055887)]"


In [5]:
corpus['뉴스제목'] = corpus['뉴스제목'].str.replace('\s+', ' ', regex=True)
corpus['뉴스제목'] = corpus['뉴스제목'].map(lambda x: re.sub('[0-9,.!·,…*°>%@#&$<_?.:\“\/\”\’\'\"\^\\‘\|\`\(\)\[\]\{\}]', '', x))
corpus['뉴스제목'] = corpus['뉴스제목'].map(lambda x: re.sub('\u200c', '', x))
corpus['뉴스제목'] = corpus['뉴스제목'].map(lambda x: re.sub('[-–ㆍ]', ' ', x))

stop_words = ['\u200c','\n ','\n',"'",'…',',','[',']','(',')','"','주','에','·','장','-','적',\
              '도','‘','`','가','’','의','이','★','은','“','대','”','한','B','로',\
              '?','선','A','는','!','"…','상','들','제','…"','일','서','명',"'…",'기',\
              '···','소','등','자','전','률','미','...','세','시','안','폭',"…'",'만','억',\
              '눈','더','량','고','인','성','다','감','을','지','수','것','째',\
              '기','···','중','계','왜','총','내','과','젠','또','연','엔','차','할',\
              '새','사','때','..','임','속','’…','G','나','개','원',\
             '달','→','권','?…','간','배','K','저','와','하','/','조','두','분','형',\
             '황','공','&','보','문','익','X','억원',']"','치','산','를','오','해','S','그','된','준','▶',\
             '건','재','반','라','년','초','분','월','신','p','급','줄','경','구','진','올','발','vs','강',\
             '국','난','판','면','"(','`…','살','아','번','텍','팜','Q','메','점','월',\
             'D','비','됐다','채',"]'",'보니','손','확','종','동','팔','타','~','땐','말','요',\
             "',",'스','…`','단','길','회','호','용','듯','최']

corpus['뉴스제목'] = [each_word for each_word in corpus['뉴스제목'] if each_word not in stop_words]

In [6]:
without_dif = corpus['주가변동'].value_counts()[0] - corpus['주가변동'].value_counts()[1]
pd_without = corpus.drop(corpus[corpus['주가변동'] == 0].sample(n=without_dif).index)
print('Number of Price up: {}'.format(corpus['주가변동'].value_counts()[1]))
print('Number of Price down: {}'.format(corpus['주가변동'].value_counts()[0]))

Number of Price up: 285481
Number of Price down: 325992


In [29]:
tCorpus = corpus[corpus["가장 비중이 높은 토픽"] == corpus["주가변동"]]

In [30]:
without_dif = tCorpus['주가변동'].value_counts()[0] - tCorpus['주가변동'].value_counts()[1]
pd_without = tCorpus.drop(tCorpus[tCorpus['주가변동'] == 0].sample(n=without_dif).index)
print('Number of Price up: {}'.format(tCorpus['주가변동'].value_counts()[1]))
print('Number of Price down: {}'.format(tCorpus['주가변동'].value_counts()[0]))

Number of Price up: 124255
Number of Price down: 180822


In [31]:
tCorpus['str_len']= tCorpus['뉴스제목'].str.len()
tCorpus.loc[(tCorpus['str_len'] < 10)]

,뉴스제목,주가변동,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중,str_len
2345,인사OCI,0,2345,0,0.8302,"[(0, 0.83021593), (1, 0.1697841)]",5
2754,인사LG디스플레이,0,2754,0,0.8503,"[(0, 0.8502759), (1, 0.1497241)]",9
2891,인사 한국전력,0,2891,0,0.8333,"[(0, 0.83329886), (1, 0.16670112)]",7
4252,인사 대한전선,0,4252,0,0.8290,"[(0, 0.82901806), (1, 0.17098191)]",7
4273,인사메리츠화재,0,4273,0,0.8332,"[(0, 0.8332452), (1, 0.16675472)]",7
...,...,...,...,...,...,...,...
607609,인사 NH투자증권,0,607609,0,0.9000,"[(0, 0.89999586), (1, 0.100004114)]",9
608494,인사 셀트리온그룹,0,608494,0,0.6250,"[(0, 0.624986), (1, 0.37501398)]",9
608720,인사 우리금융지주,0,608720,0,0.8750,"[(0, 0.8749694), (1, 0.12503058)]",9
608977,인사 한미약품그룹,0,608977,0,0.5532,"[(0, 0.5531815), (1, 0.44681847)]",9


In [ ]:

indexnames = tCorpus[tCorpus['str_len'] <= 10].index
tCorpus.drop(indexnames, inplace = True)
tCorpus.loc[(tCorpus['str_len'] < 10)]

In [36]:
tCorpus.loc[(tCorpus['str_len'] < 12)]

,뉴스제목,주가변동,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중,str_len
757,SK하이닉스의 반도체,0,757,0,0.8618,"[(0, 0.8617779), (1, 0.13822211)]",11
15031,기아 EV 모델 공개,0,15031,0,0.6103,"[(0, 0.6103205), (1, 0.38967946)]",11
20048,셀리버리 특허권 취득,1,20048,1,0.8999,"[(0, 0.10005634), (1, 0.8999436)]",11
20149,대웅 + 상승폭 확대,1,20149,1,0.7033,"[(0, 0.2967067), (1, 0.70329326)]",11
21631,LG전자 지스타 참가,1,21631,1,0.8472,"[(0, 0.15278547), (1, 0.8472145)]",11
...,...,...,...,...,...,...,...
603565,셀리버리 특허권 취득,1,603565,1,0.8999,"[(0, 0.10005634), (1, 0.8999436)]",11
603696,CJ 현금 배당 결정,1,603696,1,0.8912,"[(0, 0.1087832), (1, 0.8912168)]",11
604089,한국조선해양 일 IR,0,604089,0,0.8752,"[(0, 0.8751965), (1, 0.124803446)]",11
610404,후성 + 상승폭 확대,1,610404,1,0.7032,"[(0, 0.29684135), (1, 0.7031587)]",11


In [37]:
without_dif = tCorpus['주가변동'].value_counts()[0] - tCorpus['주가변동'].value_counts()[1]
pd_without = tCorpus.drop(tCorpus[tCorpus['주가변동'] == 0].sample(n=without_dif).index)
print('Number of Price up: {}'.format(tCorpus['주가변동'].value_counts()[1]))
print('Number of Price down: {}'.format(tCorpus['주가변동'].value_counts()[0]))

Number of Price up: 124141
Number of Price down: 179827


In [38]:
diff = tCorpus['주가변동'].value_counts()[0] - tCorpus['주가변동'].value_counts()[1]
diff

55686

In [39]:
sCorpus = tCorpus.drop(tCorpus[tCorpus['주가변동'] == 0].sample(n=diff).index)


In [41]:
print('Number of Price up: {}'.format(sCorpus['주가변동'].value_counts()[1]))
print('Number of Price down: {}'.format(sCorpus['주가변동'].value_counts()[0]))

Number of Price up: 124141
Number of Price down: 124141


In [42]:
with open('./with_name/LDA_clean_scaled_final.pkl', 'wb') as f:
    pickle.dump(sCorpus, f)